<a href="https://colab.research.google.com/github/Rishabh672003/Learning-machine-learning/blob/main/jupyter-notebooks/fetal_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the data and pre-processing it

In [10]:
!pip install -q wfdb

In [11]:
!curl -LO https://physionet.org/static/published-projects/ctu-uhb-ctgdb/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38.2M  100 38.2M    0     0   672k      0  0:00:58  0:00:58 --:--:--  673k


In [12]:
!unzip /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip
!rm /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip

Archive:  /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0.zip
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1296.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1277.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1499.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1432.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1006.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/2023.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1294.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1336.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1302.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1219.hea  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/1218.dat  
 extracting: ctu-chb-intrapartum-cardiotocography-database-1.0.0/2031.hea  
 extracting: 

In [13]:
!git clone https://github.com/fabiom91/CTU-CHB_Physionet.org.git

fatal: destination path 'CTU-CHB_Physionet.org' already exists and is not an empty directory.


In [14]:
!mkdir /content/CTU-CHB_Physionet.org/physionet_data
!mkdir -p signals
!cp -r /content/ctu-chb-intrapartum-cardiotocography-database-1.0.0/* /content/CTU-CHB_Physionet.org/physionet_data

mkdir: cannot create directory ‘/content/CTU-CHB_Physionet.org/physionet_data’: File exists


In [15]:
import pandas as pd
import wfdb
from os import listdir
from tqdm import tqdm
import numpy as np

def get_all_records():
    rec_list = []
    for file in listdir("/content/CTU-CHB_Physionet.org/physionet_data"):
        rec = file[:file.find('.')]
        try:
            rec = int(rec)
            rec_list.append(rec)
        except:
            pass
    rec_list = [str(i) for i in rec_list]
    return rec_list

def create_signals_database(rec):
    sample = wfdb.rdsamp("/content/CTU-CHB_Physionet.org/physionet_data/%s" % rec)
    df = pd.DataFrame(sample[0], columns=['FHR','UC'])
    df.index.name = 'seconds'
    df.to_csv('signals/%s.csv' % rec)

def create_ann_dataframe(rec):
    sample = wfdb.rdsamp("/content/CTU-CHB_Physionet.org/physionet_data/%s" % rec)
    ann = sample[1]['comments'][1:]
    ann_dic = {}
    for a in ann:
        if '--' in a:
            ann.remove(a)

    for a in ann:
        key = a[:a.find('  ')]
        if a.find('  ') == -1:
            key = a[:a.find(' ')]
        inv = a[::-1]
        value = inv[:inv.find(' ')][::-1]
        value = float(value)
        ann_dic[key] = [value]

    df1 = pd.DataFrame.from_dict(ann_dic).T
    df1 = df1.rename(columns={0:rec})
    return df1

def append_ann_dataframes(df,df1):
    rec = df1.columns[0]
    df[rec] = df1[rec]
    return df


df = pd.DataFrame()
for rec in tqdm(get_all_records()):
    create_signals_database(rec)
    df = append_ann_dataframes(df,create_ann_dataframe(rec))
df.to_csv('ann_db.csv')

print('DONE!')

 10%|▉         | 105/1104 [00:04<00:42, 23.44it/s]<ipython-input-15-241d791dcf73>:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
 10%|▉         | 108/1104 [00:04<00:42, 23.62it/s]<ipython-input-15-241d791dcf73>:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
<ipython-input-15-241d791dcf73>:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.con

DONE!


# Pre-pre-processing the dataset

In [16]:
import pandas as pd

In [17]:
df = pd.read_csv("/content/ann_db.csv")

In [18]:
df.head()

,Unnamed: 0,1128,1303,1384,1043,1024,1497,1195,1354,1049,...,1442,1119,1092,1039,1310,1228,1046,1336,1425,1001
0,pH,7.16,7.14,7.24,7.27,7.43,7.28,7.30,7.28,7.24,...,7.28,7.34,7.26,7.11,7.16,7.20,7.29,7.31,7.03,7.14
1,BDecf,4.59,5.83,3.51,3.12,0.91,-0.38,6.19,-0.70,7.28,...,0.58,5.02,2.53,8.05,6.76,5.41,4.45,2.22,7.84,8.14
2,pCO2,8.70,8.70,7.20,6.70,4.60,7.60,5.20,7.70,5.90,...,7.30,4.90,7.10,8.50,7.80,7.40,5.90,6.20,11.00,7.70
3,BE,-7.10,-8.40,-5.30,-4.80,-1.30,-1.30,-7.30,-0.90,-8.30,...,-3.00,-5.80,-4.40,-11.00,-8.70,-7.30,-5.60,-3.90,-10.70,-10.50
4,Apgar1,8.00,8.00,9.00,8.00,9.00,10.00,9.00,10.00,8.00,...,9.00,10.00,9.00,8.00,8.00,9.00,9.00,9.00,6.00,6.00


In [21]:
df = df.T

In [22]:
df.shape

(553, 35)

In [25]:
df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
Unnamed: 0,pH,BDecf,pCO2,BE,Apgar1,Apgar5,NICU days,Seizures,HIE,Intubation,...,Induced,I.stage,NoProgress,CK/KP,II.stage,Deliv. type,dbID,Rec. type,Pos. II.st.,Sig2Birth
1128,7.16,4.59,8.7,-7.1,8.0,10.0,0.0,0.0,0.0,0.0,...,0.0,215.0,1.0,0.0,15.0,1.0,1030266.0,1.0,14400.0,0.0
1303,7.14,5.83,8.7,-8.4,8.0,9.0,0.0,0.0,0.0,0.0,...,0.0,363.0,0.0,0.0,10.0,1.0,1065456.0,1.0,14400.0,0.0
1384,7.24,3.51,7.2,-5.3,9.0,9.0,0.0,0.0,0.0,0.0,...,0.0,375.0,0.0,0.0,30.0,1.0,1079677.0,1.0,14400.0,0.0
1043,7.27,3.12,6.7,-4.8,8.0,9.0,0.0,0.0,0.0,0.0,...,1.0,280.0,0.0,0.0,10.0,1.0,1006412.0,1.0,14400.0,0.0
1024,7.43,0.91,4.6,-1.3,9.0,10.0,0.0,0.0,0.0,0.0,...,1.0,85.0,0.0,0.0,5.0,1.0,993822.0,1.0,14400.0,0.0
1497,7.28,-0.38,7.6,-1.3,10.0,10.0,0.0,0.0,0.0,0.0,...,0.0,164.0,0.0,0.0,10.0,1.0,1115719.0,1.0,14400.0,0.0
1195,7.3,6.19,5.2,-7.3,9.0,9.0,0.0,0.0,0.0,0.0,...,1.0,235.0,0.0,0.0,10.0,1.0,1046876.0,2.0,14400.0,0.0
1354,7.28,-0.7,7.7,-0.9,10.0,10.0,0.0,0.0,0.0,0.0,...,1.0,225.0,0.0,0.0,15.0,1.0,1072051.0,1.0,14400.0,0.0
1049,7.24,7.28,5.9,-8.3,8.0,8.0,0.0,0.0,0.0,0.0,...,0.0,320.0,0.0,0.0,5.0,1.0,1008016.0,1.0,14400.0,0.0


In [26]:
import numpy as np

new_cols = pd.DataFrame(columns=['index','FHR_mean','UC_mean', "FHR_rms", "UC_rms", "FHR_max", "UC_max"])
for i in range(1001, 2046):
    f = f"signals/{i}.csv"
    try:
      df_temp = pd.read_csv(f)

      fhr_mean, uc_mean = df_temp["FHR"].mean(), df_temp["UC"].mean()
      fhr_rms = np.sqrt(np.mean(np.square(df_temp['FHR'])))
      uc_rms = np.sqrt(np.mean(np.square(df_temp['UC'])))
      fhr_max, uc_max = df_temp.loc[df_temp['FHR'].idxmax()]["FHR"], df_temp.loc[df_temp['UC'].idxmax()]["UC"]
      new_cols.loc[0] = [i-1001, fhr_mean, uc_mean, fhr_rms, uc_rms, fhr_max, uc_max]
      new_cols.index += 1
      new_cols = new_cols.sort_index()
    except:
      continue


In [27]:
new_cols.head()

,index,FHR_mean,UC_mean,FHR_rms,UC_rms,FHR_max,UC_max
1,1044.0,96.125571,14.546898,106.547922,22.618719,167.50,127.0
2,1043.0,115.084445,15.543302,133.228479,22.951402,182.00,102.5
3,1042.0,82.792328,18.059265,102.957196,28.069936,233.50,127.0
4,1041.0,90.791597,39.765486,106.856817,48.074035,160.25,96.5
5,1040.0,105.066601,16.282025,127.023054,23.284595,178.00,98.0


In [28]:
df.columns

RangeIndex(start=0, stop=35, step=1)

In [29]:
df = df.reset_index(drop=True)

In [30]:
df.columns = [x for x in df.loc[0]]
df = df[1:].reset_index(drop=True)

In [31]:
df.head()

,pH,BDecf,pCO2,BE,Apgar1,Apgar5,NICU days,Seizures,HIE,Intubation,...,Induced,I.stage,NoProgress,CK/KP,II.stage,Deliv. type,dbID,Rec. type,Pos. II.st.,Sig2Birth
0,7.16,4.59,8.7,-7.1,8.0,10.0,0.0,0.0,0.0,0.0,...,0.0,215.0,1.0,0.0,15.0,1.0,1030266.0,1.0,14400.0,0.0
1,7.14,5.83,8.7,-8.4,8.0,9.0,0.0,0.0,0.0,0.0,...,0.0,363.0,0.0,0.0,10.0,1.0,1065456.0,1.0,14400.0,0.0
2,7.24,3.51,7.2,-5.3,9.0,9.0,0.0,0.0,0.0,0.0,...,0.0,375.0,0.0,0.0,30.0,1.0,1079677.0,1.0,14400.0,0.0
3,7.27,3.12,6.7,-4.8,8.0,9.0,0.0,0.0,0.0,0.0,...,1.0,280.0,0.0,0.0,10.0,1.0,1006412.0,1.0,14400.0,0.0
4,7.43,0.91,4.6,-1.3,9.0,10.0,0.0,0.0,0.0,0.0,...,1.0,85.0,0.0,0.0,5.0,1.0,993822.0,1.0,14400.0,0.0


In [32]:
df = df.reset_index()  # Reset index to make it a column
new_cols = new_cols.rename(columns={'index': 'index'})

# Merge on 'index'
merged_df = df.merge(new_cols, on='index', how='inner')

merged_df.head()

,index,pH,BDecf,pCO2,BE,Apgar1,Apgar5,NICU days,Seizures,HIE,...,dbID,Rec. type,Pos. II.st.,Sig2Birth,FHR_mean,UC_mean,FHR_rms,UC_rms,FHR_max,UC_max
0,0,7.16,4.59,8.7,-7.1,8.0,10.0,0.0,0.0,0.0,...,1030266.0,1.0,14400.0,0.0,106.984336,24.815130,122.357020,33.143752,193.00,127.0
1,1,7.14,5.83,8.7,-8.4,8.0,9.0,0.0,0.0,0.0,...,1065456.0,1.0,14400.0,0.0,122.064076,24.777604,135.710988,34.975993,224.00,127.0
2,2,7.24,3.51,7.2,-5.3,9.0,9.0,0.0,0.0,0.0,...,1079677.0,1.0,14400.0,0.0,97.129514,14.892778,109.963878,26.323443,174.50,100.0
3,3,7.27,3.12,6.7,-4.8,8.0,9.0,0.0,0.0,0.0,...,1006412.0,1.0,14400.0,0.0,133.030074,22.084762,135.560556,37.570082,166.75,100.0
4,4,7.43,0.91,4.6,-1.3,9.0,10.0,0.0,0.0,0.0,...,993822.0,1.0,14400.0,0.0,92.547375,15.335806,105.951334,22.315540,229.00,123.0


In [34]:
merged_df.drop(["index"], axis=1)

,pH,BDecf,pCO2,BE,Apgar1,Apgar5,NICU days,Seizures,HIE,Intubation,...,dbID,Rec. type,Pos. II.st.,Sig2Birth,FHR_mean,UC_mean,FHR_rms,UC_rms,FHR_max,UC_max
0,7.16,4.59,8.7,-7.1,8.0,10.0,0.0,0.0,0.0,0.0,...,1030266.0,1.0,14400.0,0.0,106.984336,24.815130,122.357020,33.143752,193.00,127.0
1,7.14,5.83,8.7,-8.4,8.0,9.0,0.0,0.0,0.0,0.0,...,1065456.0,1.0,14400.0,0.0,122.064076,24.777604,135.710988,34.975993,224.00,127.0
2,7.24,3.51,7.2,-5.3,9.0,9.0,0.0,0.0,0.0,0.0,...,1079677.0,1.0,14400.0,0.0,97.129514,14.892778,109.963878,26.323443,174.50,100.0
3,7.27,3.12,6.7,-4.8,8.0,9.0,0.0,0.0,0.0,0.0,...,1006412.0,1.0,14400.0,0.0,133.030074,22.084762,135.560556,37.570082,166.75,100.0
4,7.43,0.91,4.6,-1.3,9.0,10.0,0.0,0.0,0.0,0.0,...,993822.0,1.0,14400.0,0.0,92.547375,15.335806,105.951334,22.315540,229.00,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,7.39,2.09,4.9,-2.9,10.0,10.0,0.0,0.0,0.0,0.0,...,1105209.0,1.0,14400.0,0.0,101.962123,28.217560,110.624265,32.176172,226.25,127.0
502,7.35,5.2,4.7,-5.7,9.0,10.0,0.0,0.0,0.0,0.0,...,1035342.0,1.0,14400.0,0.0,117.663986,7.808306,122.720699,13.644553,164.00,93.0
503,7.18,2.54,9.0,-3.4,8.0,9.0,0.0,0.0,0.0,0.0,...,1102852.0,1.0,14400.0,0.0,105.995030,25.672768,121.027884,38.961869,208.00,127.0
504,7.32,3.68,5.6,-4.4,10.0,10.0,0.0,0.0,0.0,0.0,...,1073374.0,1.0,14400.0,0.0,127.865069,11.691444,129.991798,20.086897,173.50,100.0
